<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/CODI_DEFINITIU_ANONIMITZACIO_SQL_FAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install langdetect
! pip install deep_translator
! pip install faker


import sqlite3
import pandas as pd
from langdetect import detect
from deep_translator import GoogleTranslator
from faker import Faker
import re

def traducir_texto(texto):
    """Traduce las líneas que no están en español."""
    lineas = texto.split('\n')
    lineas_traducidas = []

    for linea in lineas:
        try:
            idioma = detect(linea)
            if idioma != 'es':
                linea = GoogleTranslator(source='auto', target='es').translate(linea)
        except:
            pass  # Si no se puede detectar el idioma, se deja la línea igual

        lineas_traducidas.append(linea)

    return '\n'.join(lineas_traducidas)

In [5]:


import re
from faker import Faker
import spacy
!python -m spacy download es_core_news_sm
def anonymizar_texto(texto):
    """Anonimiza datos sensibles en el texto, incluyendo nombres detectados por NER y frases específicas."""
    fake_es = Faker('es_ES')


    # Mapa para almacenar placeholders y sus nombres falsos correspondientes
    placeholder_map = {}
    placeholder_counter = 0

    # Patrones para detectar frases específicas y su idioma correspondiente
    patterns = [(r'(Me llamo\s+)([^.,;]+)', 'es')]  # Español


    # Procesar cada patrón para reemplazar nombres con placeholders
    for pattern, lang in patterns:
        matches = list(re.finditer(pattern, texto, flags=re.IGNORECASE))
        for match in reversed(matches):  # Procesar en orden inverso para evitar cambios en los offsets
            start_name = match.start(2)
            end_name = match.end(2)
            original_name = match.group(2).strip()

            # Generar nombre falso según el idioma
            if lang == 'es':
                fake_name = fake_es.name()


            # Crear y almacenar placeholder
            placeholder = f'{{{{NAME_{placeholder_counter}}}}}'
            placeholder_map[placeholder] = fake_name
            placeholder_counter += 1

            # Reemplazar el nombre original con el placeholder
            texto = texto[:start_name] + placeholder + texto[end_name:]

    # Anonimizar números de teléfono y DNI
    texto = re.sub(r'\b\d{9}\b', lambda x: fake_es.phone_number(), texto)
    texto = re.sub(r'\b\d{8}[A-Za-z]\b', lambda x: fake_es.ssn(), texto)

    # Cargar modelo de NER en español
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(texto)
    replacements = []

    # Detectar y reemplazar nombres mediante NER
    for ent in doc.ents:
        if ent.label_ == "PER":
            start = ent.start_char
            end = ent.end_char
            fake_name = fake_es.name()
            replacements.append((start, end, fake_name))

    # Aplicar reemplazos de NER en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    # Reemplazar placeholders con nombres falsos correspondientes
    for placeholder, fake_name in placeholder_map.items():
        texto = texto.replace(placeholder, fake_name)

    return texto

def procesar_texto(texto):
    """Traduce el texto al español y luego aplica la anonimización."""
    # Asumiendo que existe una función traducir_texto definida en otro lugar
    texto_traducido = traducir_texto(texto)
    texto_anonimizado = anonymizar_texto(texto_traducido)
    return texto_anonimizado

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 34.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
# Conectar a la base de datos y crear la tabla
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS mi_tabla (
        id INTEGER PRIMARY KEY,
        body TEXT,
        secret TEXT,
        direction TEXT,
        createdAt TEXT,
        OpenchannelAccountId INTEGER,
        OpenchannelInteractionId INTEGER,
        UserId INTEGER,
        ContactId INTEGER,
        AttachmentId INTEGER,
        sentBy TEXT
    );
    """)
    conn.commit()

In [7]:
    # Cargar datos desde Excel a la base de datos
    df = pd.read_excel("/content/MOSTRA_1.xlsx")  # Reemplazar con el nombre real del archivo
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer la columna body
    df_body = pd.read_sql("SELECT id, body, createdAt, direction, ContactId FROM mi_tabla", conn)

    # Aplicar procesamiento a la columna body
    df_body['body'] = df_body['body'].apply(procesar_texto)

    # Actualizar la base de datos con los valores procesados
    for index, row in df_body.iterrows():
        cursor.execute("UPDATE mi_tabla SET body = ? WHERE id = ?", (row['body'], row['id']))

    conn.commit()

print("Base de datos creada y procesamiento completado.")

KeyboardInterrupt: 

In [ ]:
print(df_body.head())

      id                                               body  \
0  77623                                               Hola   
1  77624  Bienvenido a * Chat de apoyo emocional para jó...   
2  77625                  Me llamo Nieves Montalbán Carnero   
3  77626                                              Hola!   
4  77627  Antes de empezar a chatear, es necesario que l...   

             createdAt direction  ContactId  
0  2023-01-01 00:10:19        in        466  
1  2023-01-01 00:10:19       out        466  
2  2023-01-01 00:10:37        in        466  
3  2023-01-01 00:11:12       out        466  
4  2023-01-01 00:11:33       out        466  


In [1]:
import pandas as pd
import sqlite3
from multiprocessing import Pool
import spacy
from faker import Faker
from langdetect import detect
from deep_translator import GoogleTranslator
import re

# Cargar modelo de NER en español una sola vez
nlp = spacy.load("es_core_news_sm")
fake_es = Faker('es_ES')

def traducir_texto(texto):
    """Traduce las líneas que no están en español."""
    lineas = texto.split('\n')
    lineas_traducidas = []

    for linea in lineas:
        try:
            idioma = detect(linea)
            if idioma != 'es':
                linea = GoogleTranslator(source='auto', target='es').translate(linea)
        except:
            pass  # Si no se puede detectar el idioma, se deja la línea igual

        lineas_traducidas.append(linea)

    return '\n'.join(lineas_traducidas)

def anonymizar_texto(texto):
    """Anonimiza datos sensibles en el texto, incluyendo nombres detectados por NER y frases específicas."""
    placeholder_map = {}
    placeholder_counter = 0
    patterns = [(r'(Me llamo\s+)([^.,;]+)', 'es')]  # Español

    for pattern, lang in patterns:
        matches = list(re.finditer(pattern, texto, flags=re.IGNORECASE))
        for match in reversed(matches):
            start_name = match.start(2)
            end_name = match.end(2)
            original_name = match.group(2).strip()

            if lang == 'es':
                fake_name = fake_es.name()

            placeholder = f'{{{{NAME_{placeholder_counter}}}}}'
            placeholder_map[placeholder] = fake_name
            placeholder_counter += 1

            texto = texto[:start_name] + placeholder + texto[end_name:]

    texto = re.sub(r'\b\d{9}\b', lambda x: fake_es.phone_number(), texto)
    texto = re.sub(r'\b\d{8}[A-Za-z]\b', lambda x: fake_es.ssn(), texto)

    doc = nlp(texto)
    replacements = []

    for ent in doc.ents:
        if ent.label_ == "PER":
            start = ent.start_char
            end = ent.end_char
            fake_name = fake_es.name()
            replacements.append((start, end, fake_name))

    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    for placeholder, fake_name in placeholder_map.items():
        texto = texto.replace(placeholder, fake_name)

    return texto

def procesar_fila(row):
    body = row['body']
    body_traducido = traducir_texto(body)
    body_anonimizado = anonymizar_texto(body_traducido)
    row['body'] = body_anonimizado
    return row

# Conectar a la base de datos y cargar datos
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()
    df_body = pd.read_sql("SELECT id, body, createdAt, direction, ContactId FROM mi_tabla", conn)

    # Procesar en paralelo
    with Pool() as pool:
        df_body = pd.DataFrame(pool.map(procesar_fila, [row for _, row in df_body.iterrows()]))

    # Actualizar la base de datos con los valores procesados
    for index, row in df_body.iterrows():
        cursor.execute("UPDATE mi_tabla SET body = ? WHERE id = ?", (row['body'], row['id']))

    conn.commit()

print("Base de datos creada y procesamiento completado.")

KeyboardInterrupt: 

In [8]:
!pip install langdetect
!pip install deep_translator
!pip install faker
!pip install tqdm
!python -m spacy download es_core_news_sm

import sqlite3
import pandas as pd
import re
import time
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from langdetect import detect, DetectorFactory
from deep_translator import GoogleTranslator
from faker import Faker
import spacy

# Configuración inicial para consistencia en detección de idioma
DetectorFactory.seed = 0

# Precargar recursos compartidos
nlp = spacy.load("es_core_news_sm")
fake_es = Faker('es_ES')
translator = GoogleTranslator(source='auto', target='es')

# Precompilar expresiones regulares
PHONE_PATTERN = re.compile(r'\b\d{9}\b')
DNI_PATTERN = re.compile(r'\b\d{8}[A-Za-z]\b')
NAME_PATTERNS = [(re.compile(r'(Me llamo\s+)([^.,;]+)', flags=re.IGNORECASE), 'es')]

def traducir_texto(texto):
    """Traduce eficientemente el texto no español usando batching"""
    if not texto.strip():
        return texto

    try:
        if detect(texto) == 'es':
            return texto
    except:
        return texto

    lineas = texto.split('\n')
    traducciones = []

    # Procesar en bloques para reducir llamadas al traductor
    bloque = []
    for linea in lineas:
        try:
            if detect(linea) != 'es':
                bloque.append(linea)
            else:
                if bloque:
                    traducciones.extend(translator.translate_batch(bloque))
                    bloque = []
                traducciones.append(linea)
        except:
            traducciones.append(linea)

    if bloque:
        traducciones.extend(translator.translate_batch(bloque))

    return '\n'.join(traducciones)

def anonymizar_texto(texto):
    """Versión optimizada de anonimización con procesamiento por lotes"""
    if not texto.strip():
        return texto

    # Sustitución de números
    texto = DNI_PATTERN.sub(lambda x: fake_es.ssn(), texto)
    texto = PHONE_PATTERN.sub(lambda x: fake_es.phone_number(), texto)

    # Detección de nombres mediante patrones
    placeholder_map = {}
    for pattern, lang in NAME_PATTERNS:
        for match in reversed(list(pattern.finditer(texto))):
            original = match.group(2).strip()
            fake_name = fake_es.name()
            placeholder = f'{{NAME_{len(placeholder_map)}}}'
            placeholder_map[placeholder] = fake_name
            texto = texto[:match.start(2)] + placeholder + texto[match.end(2):]

    # Procesamiento NER por lotes
    docs = nlp.pipe([texto], disable=["tagger", "parser"])
    for doc in docs:
        replacements = []
        for ent in reversed(doc.ents):
            if ent.label_ == "PER":
                fake_name = fake_es.name()
                replacements.append((ent.start_char, ent.end_char, fake_name))

        for start, end, name in replacements:
            texto = texto[:start] + name + texto[end:]

    # Reemplazar placeholders finales
    for placeholder, name in placeholder_map.items():
        texto = texto.replace(placeholder, name)

    return texto

def procesar_fila(args):
    """Función de procesamiento para paralelismo"""
    idx, row = args
    try:
        traducido = traducir_texto(row['body'])
        anonimizado = anonymizar_texto(traducido)
        return (idx, anonimizado)
    except Exception as e:
        print(f"Error procesando fila {idx}: {str(e)}")
        return (idx, row['body'])

def procesar_dataset(df, workers=cpu_count()):
    """Procesamiento paralelo con seguimiento de progreso"""
    results = {}
    total = len(df)

    with Pool(workers) as pool:
        with tqdm(total=total, desc="Procesando registros") as pbar:
            for idx, result in pool.imap_unordered(procesar_fila, df.iterrows()):
                results[idx] = result
                pbar.update()

    # Reconstruir orden original
    df['body'] = [results[idx] for idx in sorted(results)]
    return df

def main():
    start_time = time.time()

    # Conexión y carga de datos
    with sqlite3.connect("mi_base_de_datos.db") as conn:
        # Cargar datos directamente desde Excel
        df = pd.read_excel("/content/MOSTRA_1.xlsx")
        df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

        # Procesamiento
        df = pd.read_sql("SELECT id, body FROM mi_tabla", conn)
        df = procesar_dataset(df)

        # Actualización masiva
        conn.executemany("UPDATE mi_tabla SET body = ? WHERE id = ?",
                        df[['body', 'id']].values.tolist())
        conn.commit()

    total_time = time.time() - start_time
    print(f"\nProceso completado en {total_time:.2f} segundos")

if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 48.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Procesando registros: 100%|██████████| 6943/6943 [06:03<00:00, 19.09it/s]



Proceso completado en 367.78 segundos
